In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\USFS_2020_2025.gdb"
reference_layer_name = 'usfs_edw_facts_common_attributes_enriched_Actv_CommonAttribute_PL_Region05_2023_20240918'
output_layer_name = 'USFS_Region05_enriched_20250310'

In [4]:
def val_group(reference_gdb_path, 
              output_gdb_path, 
              reference_layer_name,
              output_layer_name,
              col_sum='ACTIVITY_QUANTITY', 
              group=['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', "YEAR_TXT"]):
    
    reference_gdf = gpd.read_file(reference_gdb_path, driver='openFileGDB', layer=reference_layer_name)
    output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
    
    reference_gdf.columns = [col.upper() for col in reference_gdf.columns]
    output_gdf.columns = [col.upper() for col in output_gdf.columns]
    
    out_group = output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    ref_group = reference_gdf[reference_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    
    merge_group = out_group.merge(ref_group, on=group, how='outer')
    
    print('out of {} matching combinations of {}, there are {} combinations that have different sum activity quantity'.format(len(merge_group),
                                                                                                                              group,
                                                                                                                             len(merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]),
                                                                                                                             ))
    
    diff_df = merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]
    diff_df['DIFF'] = diff_df['ACTIVITY_QUANTITY_x'].fillna(0) - diff_df['ACTIVITY_QUANTITY_y'].fillna(0)
    
    return diff_df

In [7]:
gpd.list_layers(r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb")

,name,geometry_type
0,CNRA_enriched_20250212_polygon,MultiPolygon
1,CNRA_enriched_20250212_line,MultiLineString
2,CNRA_enriched_20250212_point,Point
3,CNRA_enriched_20250219_polygon,MultiPolygon
4,CNRA_enriched_20250219_line,MultiLineString
5,CNRA_enriched_20250219_point,Point


In [8]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb"
reference_layer_name = 'CNRA_enriched_poly_20241121'
output_layer_name = "CNRA_enriched_20250219_polygon"

In [9]:
diff_df = val_group(validation_gdb_path,
                   output_gdb_path,
                   reference_layer_name,
                   output_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


out of 391 matching combinations of ['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', 'YEAR_TXT'], there are 46 combinations that have different sum activity quantity


C:\Users\sky\AppData\Local\Temp\ipykernel_68904\3042711801.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df['DIFF'] = diff_df['ACTIVITY_QUANTITY_x'].fillna(0) - diff_df['ACTIVITY_QUANTITY_y'].fillna(0)


In [12]:
reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer=reference_layer_name)
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [13]:
len(reference_gdf)

27970

In [14]:
len(output_gdf)

27562

In [17]:
treat_poly_gdf = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\2023\CNRA_2023\CNRA_Tracker_Data_Export_20241120.gdb', driver='openFileGDB', layer='TREATMENT_POLY')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [32]:
len(treat_poly_gdf[~treat_poly_gdf['globalid_text'].notna()][['globalid_text','globalid']])

36

In [31]:
treat_poly_gdf[treat_poly_gdf['globalid_text'] != treat_poly_gdf['globalid']]

,globalid,projectid,trmtid_user,projectid_user,projectname_,org_admin_t,primary_ownership_group,primary_objective,secondary_objective,tertiary_objective,...,treatment_name,globalid_text,created_user,created_date,last_edited_user,last_edited_date,validationstatus,shape_Length,shape_Area,geometry
742,{6D2CA5B3-DA1A-4678-93AC-701556893A81},{A7D188DC-8126-4D90-BB22-92AAF550E181},21-043-01-T1,21-043-01,Carpinteria-Summerland Fire Fuels Reduction Pr...,SCC,PRIVATE_NON-INDUSTRY,OTHER_FUELS_REDUCTION,None,None,...,None,None,cnra.trmttracker.admin,2024-05-14 17:06:37+00:00,cnra.trmttracker.scc,2024-10-03 16:58:04+00:00,NaN,60000.429521,1.489516e+08,"MULTIPOLYGON (((-13296178.735 4087851.377, -13..."
743,{672E325D-3B81-436D-9239-EC227C1A88B3},{B26B4195-E30F-4CAD-B24E-AD88A5818EA8},Viejas 2023,Viejas Fuel Reduction,Viejas Fuel Reduction,SDRC,TRIBAL,OTHER_FUELS_REDUCTION,FUEL_BREAK,,...,Viejas 2023,None,cnra.tracker2.sdrc,2024-07-23 21:27:52+00:00,cnra.tracker2.sdrc,2024-07-23 21:34:58+00:00,NaN,8490.341367,4.696096e+05,"MULTIPOLYGON (((-12991469.667 3875461.982, -12..."
744,{9916398E-7925-4B47-8546-0941C8514CFE},{05C47BCE-959A-4606-B23A-A656937F6B76},Silverwood,SD Audubon Society,Silverwood Wildlife Sanctuary,SDRC,NGO,OTHER_FUELS_REDUCTION,FUEL_BREAK,,...,Silverwood Fuels Reduction,None,cnra.tracker2.sdrc,2024-07-29 20:13:00+00:00,cnra.tracker2.sdrc,2024-07-29 21:07:38+00:00,NaN,4524.517959,6.976162e+05,"MULTIPOLYGON (((-13011286.255 3884620.371, -13..."
746,{380746FA-A8DA-4430-A231-EEB89DE18EBB},{92418BFF-8595-43D9-AE21-FCB8A9220CAC},2024 ranchosimi_vegmgmtoak barrier trtmt,2024 ranchosimi_vegmgmtoak barrier,2024 ranchosimi_vegmgmtoak barrier,SMMC,LOCAL,FIRE_PREVENTION,None,None,...,2024 ranchosimi_vegmgmtoak barrier trtmt,None,cnra.tracker2.smmc,2024-08-12 18:41:11+00:00,cnra.tracker2.smmc,2024-08-22 18:26:34+00:00,NaN,8170.607156,1.934055e+05,"MULTIPOLYGON (((-13218897.094 4055258.723, -13..."
747,{B248A9EA-D102-46C0-89B4-DEF5B1525AB9},{06F7C539-88A9-40CF-A660-992684201DFD},2024 moorpark_firebufferoak,2024 cityofmoorpark_firebuffer,2024 moorpark_firebufferoak name,SMMC,LOCAL,HABITAT_RESTOR,FIRE_PREVENTION,None,...,2024 moorpark_firebufferoak trtmt,None,cnra.tracker2.smmc,2024-08-12 18:47:00+00:00,cnra.tracker2.smmc,2024-09-01 18:22:44+00:00,NaN,1197.128817,1.120946e+04,"MULTIPOLYGON (((-13232086.022 4064105.476, -13..."
748,{12F82089-8AAE-4097-BBEF-5140CC0F584B},{F8FCCC4A-084B-4278-8453-16D4581C3A01},Kanan Dume Roadside Clearance,2024 Kanan Dume\n,2024 Kanan Dume,SMMC,LOCAL,FIRE_PREVENTION,None,None,...,2024 Kanan Dume\n Roadside Clearance,None,cnra.tracker2.smmc,2024-08-19 23:49:39+00:00,cnra.tracker2.smmc,2024-08-23 22:01:44+00:00,NaN,5093.647149,2.295242e+04,"MULTIPOLYGON (((-13224255.856 4036559.781, -13..."
749,{23F2849E-E124-4707-B5AA-4911D2668376},{7639D490-80CC-47FF-B2F9-854ADC88D31C},,,San Diego County Arundo Removal,,,,,,...,,None,cnra.tracker2.sdrc,2024-08-26 22:10:56+00:00,cnra.tracker2.sdrc,2024-08-26 22:10:56+00:00,NaN,475.840644,9.775254e+03,"MULTIPOLYGON (((-13035886.094 3867641.708, -13..."
750,{88C1F8F5-16A4-4A41-AED7-570AC40AA516},{7639D490-80CC-47FF-B2F9-854ADC88D31C},Nature Collective,The Nature Collective,San Diego County Arundo Removal,SDRC,PRIVATE_NON-INDUSTRY,OTHER_FUELS_REDUCTION,,,...,Arundo Removal,None,cnra.tracker2.sdrc,2024-08-26 22:10:56+00:00,cnra.tracker2.sdrc,2024-08-26 22:13:17+00:00,NaN,822.427019,1.324551e+04,"MULTIPOLYGON (((-13035835.275 3867143.051, -13..."
751,{A09AB99E-3E94-4749-ADD0-AE95798FC741},{EFD6AAF9-1135-4574-9DC1-D18016971C04},San Diego Girl Scouts,Camp Winacka/Whispering Oaks,Camp Winacka/Camp Whispering Oaks,SDRC,NGO,PRESCRB_FIRE,OTHER_FUELS_REDUCTION,,...,Curtain Burner,None,cnra.tracker2.sdrc,2024-08-26 22:18:59+00:00,cnra.tracker2.sdrc,2024-08-26 22:21:43+00:00,NaN,2924.059250,3.430647e+05,"MULTIPOLYGON (((-12984719.611 3899842.376, -12..."
752,{A3B13A19-2CE3-4F54-BA94-4D0AB2084C2E},{FD79F7BF-269E-4E0E-82A6-30CD8A0E47D5},Campo,Campo Band of Mission Indians,Campo Wildfire Protect

In [26]:
len(treat_poly_gdf)

11165

In [10]:
diff_df

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y,DIFF
101,CCC,FOREST,OTHER_FUELS_REDUCTION,2025,3.510000,NaN,3.510000
102,CCC,FOREST,OTHER_FUELS_REDUCTION,2027,NaN,3.510000,-3.510000
103,CCC,FOREST,TBD,2023,NaN,2.250000,-2.250000
113,CCC,SHRB_CHAP,OTHER_FUELS_REDUCTION,2025,10.990000,NaN,10.990000
114,CCC,SHRB_CHAP,OTHER_FUELS_REDUCTION,2027,NaN,10.990000,-10.990000
119,CCC,URBAN,OTHER_FUELS_REDUCTION,2025,11.290000,NaN,11.290000
120,CCC,URBAN,OTHER_FUELS_REDUCTION,2027,NaN,11.290000,-11.290000
134,CDFW,GRASS_HERB,BURNED_AREA_RESTOR,2023,11.000000,23.000000,-12.000000
135,CDFW,GRASS_HERB,BURNED_AREA_RESTOR,2024,8.000000,28.000000,-20.000000
137,CDFW,GRASS_HERB,FUEL_BREAK,2022,5160.588274,5162.088274,-1.500000


In [13]:
sum(diff_df['DIFF'])

-12769.1

In [12]:
diff_df[diff_df['YEAR_TXT'] == '2023']

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y,DIFF
1,USFS,AGRICULTURE,ECO_RESTOR,2023,NaN,10.9,-10.9
14,USFS,FOREST,ECO_RESTOR,2023,1.0,214.8,-213.8
54,USFS,GRASS_HERB,ECO_RESTOR,2023,NaN,41.6,-41.6
77,USFS,SHRB_CHAP,ECO_RESTOR,2023,458.0,632.3,-174.3
113,USFS,SPARSE,ECO_RESTOR,2023,NaN,76.4,-76.4
126,USFS,URBAN,ECO_RESTOR,2023,4.0,6.3,-2.3
141,USFS,WATER,ECO_RESTOR,2023,1.0,2.2,-1.2


In [28]:
113038/sum(output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES']['ACTIVITY_QUANTITY'])

0.16159465756093824

In [25]:
diff_df

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,Year_txt,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y,DIFF
16,USFS,FOREST,FUEL_BREAK,2020,1391.0,1300.0,91.0
29,USFS,FOREST,OTHER_FUELS_REDUCTION,2020,98310.6,108060.1,-9749.5
30,USFS,FOREST,OTHER_FUELS_REDUCTION,2021,79961.7,82934.5,-2972.8
31,USFS,FOREST,OTHER_FUELS_REDUCTION,2022,80280.5,82152.5,-1872.0
32,USFS,FOREST,OTHER_FUELS_REDUCTION,2023,106991.5,139945.5,-32954.0
33,USFS,FOREST,OTHER_FUELS_REDUCTION,2024,17099.5,76232.7,-59133.2
36,USFS,FOREST,PRESCRB_FIRE,2022,2557.8,2524.8,33.0
62,USFS,GRASS_HERB,OTHER_FUELS_REDUCTION,2020,978.8,1486.2,-507.4
90,USFS,SHRB_CHAP,OTHER_FUELS_REDUCTION,2020,5498.3,11291.3,-5793.0
92,USFS,SHRB_CHAP,OTHER_FUELS_REDUCTION,2022,7085.3,7199.3,-114.0


In [20]:
out_rmc_ofr=out_rmc[out_rmc.PRIMARY_OBJECTIVE == 'OTHER_FUELS_REDUCTION']
out_rmc_ofr_2022 = out_rmc_ofr[out_rmc_ofr.Year_txt=='2022']

In [21]:
pd.set_option('display.max_columns', None)
out_rmc_ofr_2022 = out_rmc_ofr_2022[out_rmc_ofr_2022.COUNTS_TO_MAS=='YES']

In [22]:
out_rmc_ofr_2022['PROJECTID_USER'].unique()

array(['40329-CNRA', 'RMC21027-CNRA'], dtype=object)

In [23]:
out_rmc_ofr_2022

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,PRIMARY_FUNDING_SOURCE,PRIMARY_FUNDING_ORG,IMPLEMENTING_ORG,LATITUDE,LONGITUDE,Val_Status_p,Val_Message_p,Val_RunDate_p,Review_Status_p,Review_Message_p,Review_RunDate_p,Dataload_Status_p,Dataload_Msg_p,TRMTID_USER,PROJECTID,PROJECTNAME_,ORG_ADMIN_t,PRIMARY_OWNERSHIP_GROUP,PRIMARY_OBJECTIVE,SECONDARY_OBJECTIVE,TERTIARY_OBJECTIVE,TREATMENT_STATUS,COUNTY,IN_WUI,REGION,TREATMENT_AREA,TREATMENT_START,TREATMENT_END,RETREATMENT_DATE_EST,TREATMENT_NAME,Val_Status_t,Val_Message_t,Val_RunDate_t,Review_Status_t,Review_Message_t,Review_RunDate_t,Dataload_Status_t,Dataload_Msg_t,ACTIVID_USER,TREATMENTID_,ORG_ADMIN_a,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,BVT_USERD,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_START,ACTIVITY_END,ADMIN_ORG_NAME,IMPLEM_ORG_NAME,PRIMARY_FUND_SRC_NAME,PRIMARY_FUND_ORG_NAME,SECONDARY_FUND_SRC_NAME,SECONDARY_FUND_ORG_NAME,TERTIARY_FUND_SRC_NAME,TERTIARY_FUND_ORG_NAME,ACTIVITY_PRCT,RESIDUE_FATE,RESIDUE_FATE_QUANTITY,RESIDUE_FATE_UNITS,ACTIVITY_NAME,VAL_STATUS_a,VAL_MSG_a,VAL_RUNDATE_a,REVIEW_STATUS_a,REVIEW_MSG_a,REVIEW_RUNDATE_a,DATALOAD_STATUS_a,DATALOAD_MSG_a,Source,Year,Year_txt,Act_Code,Crosswalk,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,geometry
6,40329-CNRA,CNRA,RMC,Michelle Mariscal,mmariscal@habitatauthority.org,RMC,Puente Hills Fuels Reduction and Wildfire Prev...,COMPLETE,2021-06-21 00:00:00+00:00,2022-10-30 00:00:00+00:00,None,RMC,Puente Hills Habitat Preservation Authority,33.972634,-117.969247,None,None,NaT,None,None,NaT,None,None,None,{6ACB04AC-620F-400E-9EA5-D61C8F155177},Puente Hills Fuels Reduction and Wildfire Prev...,RMC,LOCAL,OTHER_FUELS_REDUCTION,None,None,COMPLETE,LA,WUI_AUTO_POP,SOUTHERN_CA,0.182326,2021-06-20 07:00:00+00:00,2022-10-30 07:00:00+00:00,NaT,LH Fuel Mod Zone 1.1 Mowed,None,None,NaT,None,None,NaT,None,None,61080,{10E96520-F884-45F4-814B-011268099357},RMC,MOWING,MECH_HFR,FOREST,YES,ACTIVE,0.180,AC,2022-07-02,2022-10-30 00:00:00+00:00,RMC,Puente Hills Habitat Preservation Authority,GENERAL_FUND_SB85_2021,RMC,None,None,None,None,NaN,LEFT_ON_SITE,NaN,None,LH Fuel Mod Zone 1.1 Mowed,None,None,NaT,None,None,NaT,None,None,CNRA,2022.0,2022,0.0,MOWING,2023,2023,POLYGON,YES,"MULTIPOLYGON (((187639.402 -447199.962, 187636..."
7,40329-CNRA,CNRA,RMC,Michelle Mariscal,mmariscal@habitatauthority.org,RMC,Puente Hills Fuels Reduction and Wildfire Prev...,COMPLETE,2021-06-21 00:00:00+00:00,2022-10-30 00:00:00+00:00,None,RMC,Puente Hills Habitat Preservation Authority,33.972344,-117.967509,None,None,NaT,None,None,NaT,None,None,None,{6ACB04AC-620F-400E-9EA5-D61C8F155177},Puente Hills Fuels Reduction and Wildfire Prev...,RMC,LOCAL,OTHER_FUELS_REDUCTION,None,None,COMPLETE,LA,WUI_AUTO_POP,SOUTHERN_CA,0.509924,2021-06-20 07:00:00+00:00,2022-10-30 07:00:00+00:00,NaT,LH Fuel Mod Zone 1.2 Mowed,None,None,NaT,None,None,NaT,None,None,61081,{B692335D-D410-42A7-9CCB-B51E3C5C3CEF},RMC,MOWING,MECH_HFR,FOREST,YES,ACTIVE,0.510,AC,2022-07-02,2022-10-30 00:00:00+00:00,RMC,Puente Hills Habitat Preservation Authority,GENERAL_FUND_SB85_2021,RMC,None,None,None,None,NaN,LEFT_ON_SITE,NaN,None,LH Fuel Mod Zone 1.2 Mowed,None,None,NaT,None,None,NaT,None,None,CNRA,2022.0,2022,0.0,MOWING,2023,2023,POLYGON,YES,"MULTIPOLYGON (((187783.728 -447197.027, 187831..."
8,40329-CNRA,CNRA,RMC,Michelle Mariscal,mmariscal@habitatauthority.org,RMC,Puente Hills Fuels Reduction and Wildfire Prev...,COMPLETE,2021-06-21 00:00:00+00:00,2022-10-30 00:00:00+00:00,None,RMC,Puente Hills Habitat Preservation Authority,33.970905,-117.967028,None,None,NaT,None,None,NaT,None,None,None,{6ACB04AC-620F-400E-9EA5-D61C8F155177},Puente Hills Fuels Reduction and Wildfire Prev...,RMC,LOCAL,OTHER_FUELS_REDUCTION,None,None,COMPLETE,LA,WUI_AUTO_POP,SOUTHERN_CA,0.745724,2021-06-20 07:00:00+00:00,2022-10-30 07:00:00+00:00,NaT,LH Fuel Mod Zone 1.2 Mowed,None,None,NaT,None,None,NaT,None,None,61082,{198

In [24]:
overlap_cols = []
for c1 in list(reference_gdf.columns):
    for c2 in list(output_gdf.columns):
        if c1.upper() == c2.upper():
            reference_gdf = reference_gdf.rename(columns={c1: c2})
            overlap_cols.append(c2)

In [25]:
overlap_cols = ['PROJECTID',
 'TRMTID_USER',
 'PROJECTID_USER',
 'PROJECTNAME_',
 'ORG_ADMIN_t',
 'PRIMARY_OWNERSHIP_GROUP',
 'PRIMARY_OBJECTIVE',
 'SECONDARY_OBJECTIVE',
 'TERTIARY_OBJECTIVE',
 'TREATMENT_STATUS',
 'COUNTY',
 'IN_WUI',
 'REGION',
 'ACTIVID_USER',
 'TREATMENTID_',
 'ORG_ADMIN_a',
 'ACTIVITY_DESCRIPTION',
 'ACTIVITY_CAT',
 'BROAD_VEGETATION_TYPE',
 'BVT_USERD',
 'ACTIVITY_STATUS']

In [26]:
out_rmc_ofr_2022.merge(reference_gdf[reference_gdf.PROJECTID_USER.isin(['40329-CNRA', 'RMC21027-CNRA'])], on=overlap_cols, how='outer')

,PROJECTID_USER,AGENCY_x,ORG_ADMIN_p_x,PROJECT_CONTACT_x,PROJECT_EMAIL_x,ADMINISTERING_ORG_x,PROJECT_NAME_x,PROJECT_STATUS_x,PROJECT_START_x,PROJECT_END_x,PRIMARY_FUNDING_SOURCE_x,PRIMARY_FUNDING_ORG_x,IMPLEMENTING_ORG_x,LATITUDE_x,LONGITUDE_x,Val_Status_p,Val_Message_p,Val_RunDate_p,Review_Status_p,Review_Message_p,Review_RunDate_p,Dataload_Status_p,Dataload_Msg_p,TRMTID_USER,PROJECTID,PROJECTNAME_,ORG_ADMIN_t,PRIMARY_OWNERSHIP_GROUP,PRIMARY_OBJECTIVE,SECONDARY_OBJECTIVE,TERTIARY_OBJECTIVE,TREATMENT_STATUS,COUNTY,IN_WUI,REGION,TREATMENT_AREA_x,TREATMENT_START_x,TREATMENT_END_x,RETREATMENT_DATE_EST_x,TREATMENT_NAME_x,Val_Status_t,Val_Message_t,Val_RunDate_t,Review_Status_t,Review_Message_t,Review_RunDate_t,Dataload_Status_t,Dataload_Msg_t,ACTIVID_USER,TREATMENTID_,ORG_ADMIN_a,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,BVT_USERD,ACTIVITY_STATUS,ACTIVITY_QUANTITY_x,ACTIVITY_UOM_x,ACTIVITY_START_x,ACTIVITY_END_x,ADMIN_ORG_NAME_x,IMPLEM_ORG_NAME_x,PRIMARY_FUND_SRC_NAME_x,PRIMARY_FUND_ORG_NAME_x,SECONDARY_FUND_SRC_NAME_x,SECONDARY_FUND_ORG_NAME_x,TERTIARY_FUND_SRC_NAME_x,TERTIARY_FUND_ORG_NAME_x,ACTIVITY_PRCT_x,RESIDUE_FATE_x,RESIDUE_FATE_QUANTITY_x,RESIDUE_FATE_UNITS_x,ACTIVITY_NAME_x,VAL_STATUS_a_x,VAL_MSG_a_x,VAL_RUNDATE_a_x,REVIEW_STATUS_a_x,REVIEW_MSG_a_x,REVIEW_RUNDATE_a_x,DATALOAD_STATUS_a_x,DATALOAD_MSG_a_x,Source_x,Year_x,Year_txt_x,Act_Code_x,Crosswalk_x,Federal_FY_x,State_FY_x,TRMT_GEOM_x,COUNTS_TO_MAS_x,geometry_x,globalid,TREATMENT_AREA_y,TREATMENT_START_y,TREATMENT_END_y,RETREATMENT_DATE_EST_y,TREATMENT_NAME_y,ACTIVITY_QUANTITY_y,ACTIVITY_UOM_y,ACTIVITY_START_y,ACTIVITY_END_y,ADMIN_ORG_NAME_y,IMPLEM_ORG_NAME_y,PRIMARY_FUND_SRC_NAME_y,PRIMARY_FUND_ORG_NAME_y,SECONDARY_FUND_SRC_NAME_y,SECONDARY_FUND_ORG_NAME_y,TERTIARY_FUND_SRC_NAME_y,TERTIARY_FUND_ORG_NAME_y,ACTIVITY_PRCT_y,RESIDUE_FATE_y,RESIDUE_FATE_QUANTITY_y,RESIDUE_FATE_UNITS_y,ACTIVITY_NAME_y,VAL_STATUS_a_y,VAL_MSG_a_y,VAL_RUNDATE_a_y,REVIEW_STATUS_a_y,REVIEW_MSG_a_y,REVIEW_RUNDATE_a_y,DATALOAD_STATUS_a_y,DATALOAD_MSG_a_y,Source_y,Year_y,Year_txt_y,Act_Code_y,Crosswalk_y,Federal_FY_y,State_FY_y,BATCHID_a,TRMT_GEOM_y,COUNTS_TO_MAS_y,AGENCY_y,ORG_ADMIN_p_y,PROJECT_CONTACT_y,PROJECT_EMAIL_y,ADMINISTERING_ORG_y,PROJECT_NAME_y,PROJECT_STATUS_y,PROJECT_START_y,PROJECT_END_y,PRIMARY_FUNDING_SOURCE_y,PRIMARY_FUNDING_ORG_y,IMPLEMENTING_ORG_y,LATITUDE_y,LONGITUDE_y,shape_Length,shape_Area,geometry_y
0,RMC21027-CNRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,GG2024_1-CNRA,{32A4A111-DC2E-47C1-9858-B9563A741B47},Puente Hills Fuels Reduction and Wildfire Prev...,RMC,LOCAL,OTHER_FUELS_REDUCTION,None,None,COMPLETE,LA,WUI_AUTO_POP,None,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,None,None,RMC,TBD,NOT_DEFINED,GRASS_HERB,NO,COMPLETE,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,{FBE6A276-5478-4892-944A-C977A4D5372D},4.617541,2024-06-18 00:00:00+00:00,2024-06-29 00:00:00+00:00,NaT,Grazing area year 2 poly1,NaN,None,NaT,2028-12-12 00:00:00+00:00,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,NaT,None,None,NaT,None,None,CNRA,2028.0,2028,NaN,None,NaN,NaN,None,POLYGON,NO,CNRA,RMC,Michelle Mariscal,mmariscal@habitatauthority.org,RMC,Puente Hills Fuels Reduction and Wildfire Prev...,COMPLETE,2022-06-01 00:00:00+00:00,2024-12-31 00:00:00+00:00,GENERAL_FUND_SB85_2021,RMC,Puente Hills Habitat Preservation Authority,33.972721,-117.943304,1160.162831,18686.601474,"MULTIPOLYGON (((190006.366 -447005.167, 190082..."
1,RMC21027-CNRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,GG2024_2-CNRA,{32A4A111-DC2E-47C1-9858-B9563A741B47},Puente Hills Fuels Reduction and Wildfire Prev...,RMC,LOCAL,OTHER_FUELS_REDUCTION,None,None,COMPLETE,LA,WUI_AUTO_POP,None,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,None,None,RMC,TBD,NOT_DEFINED,GRASS_HERB,NO,COMPLETE,NaN,NaN,NaN,NaT,NaN